<a href="https://colab.research.google.com/github/vishalkandukuri97/AI-ML-Projects/blob/main/Realestate_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio

In [2]:
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [4]:
california = fetch_california_housing()
X = pd.DataFrame(california.data, columns=california.feature_names)
y = california.target

In [5]:
model = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(f_regression, k=5)),
    ('regressor', LinearRegression())
])

In [7]:
model.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('selector',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x7df081bde840>)),
                ('regressor', LinearRegression())])

In [8]:
selected_features = X.columns[model.named_steps['selector'].get_support()]
coefficients = model.named_steps['regressor'].coef_

In [9]:
def predict_house_price(MedInc, HouseAge, AveRooms, AveBedrms, Population, AveOccup, Latitude, Longitude):
    try:
        # Create input DataFrame with correct column order
        input_df = pd.DataFrame([[MedInc, HouseAge, AveRooms, AveBedrms, Population, AveOccup, Latitude, Longitude]],
                              columns=X.columns)

        # Make prediction
        prediction = model.predict(input_df)[0]
        price_in_dollars = prediction * 100000  # Convert to actual dollar amount

        # Create feature importance plot
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.bar(selected_features, coefficients)
        ax.set_title("Feature Impact on Price")
        ax.set_xlabel("Features")
        ax.set_ylabel("Coefficient Value")
        plt.xticks(rotation=45)
        plt.tight_layout()

        return f"${price_in_dollars:,.2f}", fig

    except Exception as e:
        print(f"Error: {str(e)}")
        return "Prediction failed", None

In [10]:
# Create Gradio interface
with gr.Blocks(title="Real Estate Housing Price Predictor") as app:
    gr.Markdown("## 🏠 California Housing Price Predictor")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### District Characteristics")
            MedInc = gr.Slider(0.5, 15, value=3.0, label="Median Income (in $10,000s)")
            HouseAge = gr.Slider(1, 52, value=28, label="House Age (years)")
            AveRooms = gr.Slider(1, 50, value=5, label="Average Rooms")
            AveBedrms = gr.Slider(0.5, 10, value=1, label="Average Bedrooms")
        with gr.Column():
            Population = gr.Slider(3, 15000, value=1500, step=100, label="Population in Block Group")
            AveOccup = gr.Slider(0.5, 10, value=3, label="Average Occupancy")
            Latitude = gr.Slider(32, 42, value=36, label="Latitude")
            Longitude = gr.Slider(-124, -114, value=-119, label="Longitude")

    submit_btn = gr.Button("Predict Price", variant="primary")

    with gr.Row():
        price_output = gr.Label(label="Predicted Median House Value")
        plot_output = gr.Plot(label="Feature Importance")

    submit_btn.click(
        fn=predict_house_price,
        inputs=[MedInc, HouseAge, AveRooms, AveBedrms, Population, AveOccup, Latitude, Longitude],
        outputs=[price_output, plot_output]
    )

In [11]:
# Launch the app
app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://04c141f399cab76154.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
